In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import re
import requests
import json

# Gather

In [2]:
# pull standard cards from mtgjson
standard_sets = requests.get('https://www.mtgjson.com/files/StandardPrintings.json')
standard_sets_clean = standard_sets.json()

# code to save standard_sets as a json file 
with open('standard_sets_clean.txt', 'w') as outfile:
    json.dump(standard_sets_clean, outfile)

# code to load standard_sets_clean
with open('standard_sets_clean.txt', encoding='utf-8') as file:
    standard_sets_clean = json.load(file)

In [ ]:
decks = pd.read_csv('C:/Users/muroc/Documents/MTG/data/decks.csv')

In [3]:
decks = pd.read_csv('/home/steadyjingo/Documents/MTGA_rarity_analysis/data/decks.csv')

In [4]:
decks_clean = decks.copy()

In [ ]:
decks_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv')

# Assess

In [ ]:
decks_clean.head()

In [ ]:
decks_clean.tail()

In [ ]:
decks_clean.sample(10)

In [6]:
decks_clean

,deck_list,player,url,name
0,"2 Shock, 10 Island, 4 Mountain, 4 Brazen Borro...",Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash
1,"4 Shock, 18 Mountain, 4 Bonecrusher Giant, 3 E...",Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins
2,"10 Island, 4 Mountain, 4 Brazen Borrower, 4 Ga...",Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control
3,"6 Forest, 3 Swamp, 4 Breeding Pool, 4 Cauldron...",Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice
4,"1 Forest, 2 Island, 2 Mountain, 4 Breeding Poo...",Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation
...,...,...,...,...
337,"1 Disenchant, 2 Island, 2 Mountain, 1 Plains, ...",Matthew_Watts,https://www.mtgtop8.com/event?e=24037&d=367686...,Jeskai_Fires
338,"4 Shock, 17 Mountain, 4 Claim the Firstborn, 4...",Sandydogmtg,https://www.mtgtop8.com/event?e=24137&d=368458...,Red_Deck_Wins
339,"4 Chandra's Spitfire, 4 Shock, 18 Mountain, 3 ...",Billy_K,https://www.mtgtop8.com/event?e=23716&d=364614...,Cavalcade
340,"9 Forest, 8 Mountain, 4 Bonecrusher Giant, 3 E...",Esther_Trujillo,https://www.mtgtop8.com/event?e=23942&d=366790...,Gruul_Aggro


In [ ]:
decks

In [ ]:
type(decks.deck_list[0])

## Assessment
- [done] deck_list column has , for delimiter but names can also have a comma
- [done] cards are in one column
- [done] multiple cards are in one object
- [done] card number and card name are the same object
- [done] null values in `deck_list` column
- [done] rarities not present in the data
- [done] some card names still have a comma at the end
- some decks have multiple observations
- [done] many cards have nan values for rarity
- [done] dual cards are considered two cards in standard_sets_clean but only one card in decks_clean

# Clean

## Define
- drop decks that have the same player and name

## Code

In [7]:
decks_clean.drop_duplicates(subset=['player', 'name'], inplace=True)

## Test

In [9]:
decks_clean.duplicated(subset=['player', 'name']).sum()

0

## Define
- filter for any lists in `deck_list` that have null values in it

## Code

In [10]:
decks_clean.deck_list = decks_clean[decks_clean.astype(str)['deck_list'] != "['nan']"]

## Test

In [11]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


## Define
- make , in `deck_list` into '|' without mistaking commas in names

## Code

In [12]:
decks_clean.deck_list = decks_clean.deck_list.astype('str')
decks_clean.deck_list = decks_clean.deck_list.str.replace(', (?=\d)', '|')

## Test

In [13]:
decks_clean.head()

,deck_list,player,url,name
0,2 Shock|10 Island|4 Mountain|4 Brazen Borrower...,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash
1,4 Shock|18 Mountain|4 Bonecrusher Giant|3 Embe...,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins
2,10 Island|4 Mountain|4 Brazen Borrower|4 Gadwi...,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control
3,6 Forest|3 Swamp|4 Breeding Pool|4 Cauldron Fa...,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice
4,1 Forest|2 Island|2 Mountain|4 Breeding Pool|2...,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation


## Define
- split `deck_list` by card

## Code

In [14]:
new = decks_clean.deck_list.str.split('|', expand=True)
decks_clean = decks_clean.join(new)

## Test

In [15]:
decks_clean.head()

,deck_list,player,url,name,0,1,2,3,4,5,...,29,30,31,32,33,34,35,36,37,38
0,2 Shock|10 Island|4 Mountain|4 Brazen Borrower...,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash,2 Shock,10 Island,4 Mountain,4 Brazen Borrower,"3 Gadwick, the Wizened",2 Mystical Dispute,...,None,None,None,None,None,None,None,None,None,None
1,4 Shock|18 Mountain|4 Bonecrusher Giant|3 Embe...,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins,4 Shock,18 Mountain,4 Bonecrusher Giant,3 Embercleave,4 Embereth Paladin,4 Rimrock Knight,...,None,None,None,None,None,None,None,None,None,None
2,10 Island|4 Mountain|4 Brazen Borrower|4 Gadwi...,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control,10 Island,4 Mountain,4 Brazen Borrower,"4 Gadwick, the Wizened",3 Mystical Dispute,4 Bonecrusher Giant,...,None,None,None,None,None,None,None,None,None,None
3,6 Forest|3 Swamp|4 Breeding Pool|4 Cauldron Fa...,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice,6 Forest,3 Swamp,4 Breeding Pool,4 Cauldron Familiar,3 Murderous Rider,4 Gilded Goose,...,None,None,None,None,None,None,None,None,None,None
4,1 Forest|2 Island|2 Mountain|4 Breeding Pool|2...,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation,1 Forest,2 Island,2 Mountain,4 Breeding Pool,2 Brazen Borrower,2 Mystical Dispute,...,None,None,None,None,None,None,None,None,None,None


## Define 
- melt cards into one column

## Code

In [16]:
decks_clean = pd.melt(decks_clean, id_vars=['deck_list', 'player', 'url', 'name'], value_vars=decks_clean.columns[4:45])

In [17]:
decks_clean = decks_clean.rename(columns={'value':'cards'}).drop(columns=['deck_list', 'variable'])

## Test

In [18]:
decks_clean.shape

(12402, 4)

In [19]:
decks_clean.head()

,player,url,name,cards
0,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash,2 Shock
1,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins,4 Shock
2,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control,10 Island
3,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice,6 Forest
4,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation,1 Forest


## Define
- drop rows that have 'None' in them

## Code

In [20]:
decks_clean.cards = decks_clean.cards.astype('str')

In [21]:
decks_clean = decks_clean[decks_clean.cards != 'None']

In [22]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Test

In [23]:
decks_clean[decks_clean.cards == 'None']

,player,url,name,cards


## Define
- get numbers out of `cards` column

## Code

In [24]:
new = decks_clean.cards.str.split('(?<=\d)\s', expand=True)
decks_clean = decks_clean.join(new).rename(columns={'name':'deck_name', 0:'num', 1:'card_name'}).drop(columns=['cards'])

## Test

In [25]:
decks_clean.head()

,player,url,deck_name,num,card_name
0,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash,2,Shock
1,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins,4,Shock
2,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control,10,Island
3,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice,6,Forest
4,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation,1,Forest


In [26]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- Add card rarity to decks_clean

In [27]:
# code to get rarity in a list by themselves
card_rarities = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        rarity = current_set[a]['rarity']
        card_rarities.append(rarity)

In [28]:
# code to get names in a list by themselves
card_names = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        name = current_set[a]['name']
        card_names.append(name)

In [29]:
# code to create dataframe of card_rarities and card_names
mtg_card_rarities = pd.DataFrame({'card_name': card_names, 'card_rarity': card_rarities})
mtg_card_rarities_clean = mtg_card_rarities.copy()
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [30]:
# drop duplicates in mtg_card_rarities_clean
mtg_card_rarities_clean.drop_duplicates(inplace=True)

In [31]:
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [32]:
mtg_card_rarities_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv')

In [33]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name')

In [34]:
decks_clean.num = decks_clean.num.astype('int64')

## Test

In [35]:
mtg_card_rarities_clean

,card_name,card_rarity
0,Acclaimed Contender,rare
1,"Alela, Artful Provocateur",mythic
2,All That Glitters,uncommon
3,Bring to Life,uncommon
4,Animating Faerie,uncommon
...,...,...
1415,Wall of Runes,common
1416,Wanderer's Strike,common
1417,War Screecher,common
1418,Wardscale Crocodile,common


In [36]:
decks_clean

,player,url,deck_name,num,card_name,card_rarity
0,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash,2,Shock,common
1,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins,4,Shock,common
2,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control,10,Island,common
3,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice,6,Forest,common
4,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation,1,Forest,common
...,...,...,...,...,...,...
11100,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Tamiyo, Collector of Tales",rare
11418,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,3,"Teferi, Time Raveler",rare
11736,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Tolsimir, Friend to Wolves",rare
12054,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,Tyrant's Scorn,uncommon


In [37]:
decks_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7586 entries, 0 to 12372
Data columns (total 6 columns):
player         7586 non-null object
url            7562 non-null object
deck_name      7562 non-null object
num            7586 non-null int64
card_name      7586 non-null object
card_rarity    7206 non-null object
dtypes: int64(1), object(5)
memory usage: 414.9+ KB


In [38]:
type(decks_clean.card_rarity[0])

str

In [39]:
decks_clean.sample(10)

,player,url,deck_name,num,card_name,card_rarity
3428,Mtg_arena_meta,https://www.mtgtop8.com/event?e=24107&d=368235...,Simic_Flash,2,Sinister Sabotage,uncommon
3572,Logan_Young,https://www.mtgtop8.com/event?e=24225&d=369066...,Adventure,3,Fabled Passage,rare
2981,Kat_Light,https://www.mtgtop8.com/event?e=23942&d=366802...,Golgari_Sacrifice,2,Castle Locthwain,rare
5149,Andrea_Gereon,https://www.mtgtop8.com/event?e=23942&d=366772...,Golgari_Sacrifice,4,Overgrown Tomb,rare
5052,Ben_Metzger,https://www.mtgtop8.com/event?e=23943&d=366835...,Azorius_Control,4,Tranquil Cove,common
6204,ACG88,https://www.mtgtop8.com/event?e=24240&d=369204...,Sultai_Ramp,4,"Nissa, Who Shakes the World",rare
8818,Parker_Connors,https://www.mtgtop8.com/event?e=23753&d=364982...,Sultai_Oven,3,"Massacre Girl,",NaN
9143,Bavan_Schimper,https://www.mtgtop8.com/event?e=24214&d=368962...,Grixis_Fires,3,"Nicol Bolas, Dragon-God,",NaN
5108,Joshua_Ng,https://www.mtgtop8.com/event?e=24144&d=368506...,Rakdos_Aggro,2,Legion's End,rare
6382,Luke_Allen,https://www.mtgtop8.com/event?e=24113&d=368280...,Fires_of_Invention,3,Shimmer of Possibility,common


In [40]:
decks_clean.card_name.nunique()

419

In [41]:
mtg_card_rarities_clean.shape

(1420, 2)

In [42]:
dc_rarity_null = decks_clean[decks_clean.card_rarity == '0']

In [43]:
dc_rarity_null.card_name.value_counts()

Series([], Name: card_name, dtype: int64)

In [44]:
dc_rarity_null.shape

(0, 6)

In [45]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- get rid of nan in `rarity`

## Code 

In [46]:
decks_clean.card_name = decks_clean.card_name.str.replace(', $', '')

In [47]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name', lsuffix='x').drop(columns='card_rarityx')

In [48]:
decks_clean.card_name = decks_clean.card_name.replace('Emry, Lurker of the Loch ', 'Emry, Lurker of the Loch')

In [49]:
conditions = [decks_clean.card_name.eq('Expansion-Explosion'), 
              decks_clean.card_name.eq('Collision-Colossus'), 
              decks_clean.card_name.eq('Flower-Flourish'), 
              decks_clean.card_name.eq('Find-Finality'), 
              decks_clean.card_name.eq('Incubation-Incongruity'), 
              decks_clean.card_name.eq('Status-Statue'), 
              decks_clean.card_name.eq('Discovery-Dispersal'), 
              decks_clean.card_name.eq('Thrash-Threat'), 
              decks_clean.card_name.eq('Emry, Lurker of the Loch')]

choices = ['rare', 'uncommon', 'uncommon', 'rare', 'uncommon', 'uncommon', 'uncommon', 'rare', 'rare']

decks_clean['rarity2'] = np.select(conditions, choices)

decks_clean.card_rarity.fillna(decks_clean.rarity2, inplace=True)

In [50]:
decks_clean.drop(columns=['rarity2'], inplace=True)

In [51]:
decks_clean = decks_clean[decks_clean.card_rarity != '0']

## Test

In [52]:
decks_clean

,player,url,deck_name,num,card_name,card_rarity
0,Jessica_Estephan,https://www.mtgtop8.com/event?e=23942&d=366798...,Izzet_Flash,2,Shock,common
1,Andrew_Boncher,https://www.mtgtop8.com/event?e=23810&d=365477...,Red_Deck_Wins,4,Shock,common
2,Brandon_Banks,https://www.mtgtop8.com/event?e=23997&d=367303...,Izzet_Control,10,Island,common
3,Andrew_C,https://www.mtgtop8.com/event?e=23716&d=364618...,Sultai_Sacrifice,6,Forest,common
4,Mx-hejji,https://www.mtgtop8.com/event?e=23796&d=365372...,Reclamation,1,Forest,common
...,...,...,...,...,...,...
11100,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Tamiyo, Collector of Tales",rare
11418,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,3,"Teferi, Time Raveler",rare
11736,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Tolsimir, Friend to Wolves",rare
12054,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,Tyrant's Scorn,uncommon


In [53]:
dc_rarity_null = decks_clean[decks_clean.card_rarity == '0']

In [54]:
dc_rarity_null.card_name.value_counts()

Series([], Name: card_name, dtype: int64)

## Define
- get rid of multiple observations

## Code

## Test

In [55]:
df = decks_clean[decks_clean.duplicated()]

In [56]:
df.query('player in "__forge__" and deck_name in "Mono_Black_Aggro" and card_name in "Massacre Girl"')

,player,url,deck_name,num,card_name,card_rarity


In [57]:
df.groupby(['player', 'deck_name', 'card_name']).num.sum()

Series([], Name: num, dtype: int64)